<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Line_Disconnects_Device_Group_Future_Demand_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [3]:
# Read a comma-separated values (csv) file into a Pandas DataFrame.
fact_table = pd.read_csv('drive/My Drive/Churn/discos_device_grp_act_vol_future_demand_jul.csv')

In [4]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,vol_port_basic_cdma,vol_port_basic_non_cdma,vol_port_sp_android_other,vol_port_sp_apple,vol_non_port_basic_cdma,vol_non_port_basic_non_cdma,vol_non_port_sp_android_other,vol_non_port_sp_apple,vol_port_tablet,vol_non_port_tablet,vol_port_watch,vol_non_port_watch,vol_port_mbb,vol_non_port_mbb,vol_port_hpc,vol_non_port_hpc,vol_port_hum,vol_non_port_hum,vol_port_gizmo,vol_non_port_gizmo,vol_port_other,vol_non_port_other
0,12/1/2016,1226,NaN,6378,6754,1511,NaN,1999,1677,112.0,5596,NaN,45.0,159.0,1591,6.0,362,NaN,86.0,NaN,146.0,11.0,21.0
1,12/2/2016,1095,NaN,6999,8020,1320,NaN,2009,1825,157.0,5785,NaN,51.0,134.0,1622,6.0,438,NaN,100.0,NaN,181.0,21.0,25.0
2,12/3/2016,1321,NaN,5232,5812,1667,NaN,1617,1477,120.0,5642,NaN,39.0,162.0,1582,7.0,376,NaN,75.0,NaN,122.0,16.0,34.0
3,12/4/2016,1451,NaN,5274,5877,1596,NaN,1603,1503,99.0,5665,NaN,26.0,164.0,1607,9.0,421,NaN,83.0,NaN,110.0,10.0,27.0
4,12/5/2016,66,NaN,432,492,53,NaN,109,94,22.0,270,NaN,NaN,4.0,61,NaN,24,NaN,6.0,NaN,8.0,2.0,4.0


In [5]:
# Split out indivudal datasets for forecasting.
# Phones - Basic CDMA
vol_port_basic_cdma = fact_table[['ds', 'vol_port_basic_cdma']].rename(columns = {'vol_port_basic_cdma': 'y'})
vol_non_port_basic_cdma = fact_table[['ds', 'vol_non_port_basic_cdma']].rename(columns = {'vol_non_port_basic_cdma': 'y'})

# Phones - Basic Non CDMA
vol_port_basic_non_cdma = fact_table[['ds', 'vol_port_basic_non_cdma']].rename(columns = {'vol_port_basic_non_cdma': 'y'})
vol_non_port_basic_non_cdma = fact_table[['ds', 'vol_non_port_basic_non_cdma']].rename(columns = {'vol_non_port_basic_non_cdma': 'y'})

# Phones - Android & Other
vol_port_sp_android_other = fact_table[['ds', 'vol_port_sp_android_other']].rename(columns = {'vol_port_sp_android_other': 'y'})
vol_non_port_sp_android_other = fact_table[['ds', 'vol_non_port_sp_android_other']].rename(columns = {'vol_non_port_sp_android_other': 'y'})

# Phones - Apple
vol_port_sp_apple = fact_table[['ds', 'vol_port_sp_apple']].rename(columns = {'vol_port_sp_apple': 'y'})
vol_non_port_sp_apple = fact_table[['ds', 'vol_non_port_sp_apple']].rename(columns = {'vol_non_port_sp_apple': 'y'})

# Tablets
vol_port_tablet = fact_table[['ds', 'vol_port_tablet']].rename(columns = {'vol_port_tablet': 'y'})
vol_non_port_tablet = fact_table[['ds', 'vol_non_port_tablet']].rename(columns = {'vol_non_port_tablet': 'y'})

# Wearables - Watches
vol_port_watch = fact_table[['ds', 'vol_port_watch']].rename(columns = {'vol_port_watch': 'y'})
vol_non_port_watch = fact_table[['ds', 'vol_non_port_watch']].rename(columns = {'vol_non_port_watch': 'y'})

# Other - MBB
vol_port_mbb = fact_table[['ds', 'vol_port_mbb']].rename(columns = {'vol_port_mbb': 'y'})
vol_non_port_mbb = fact_table[['ds', 'vol_non_port_mbb']].rename(columns = {'vol_non_port_mbb': 'y'})

# Other - HPC
vol_port_hpc = fact_table[['ds', 'vol_port_hpc']].rename(columns = {'vol_port_hpc': 'y'})
vol_non_port_hpc = fact_table[['ds', 'vol_non_port_hpc']].rename(columns = {'vol_non_port_hpc': 'y'})

# Other - Hum
vol_port_hum = fact_table[['ds', 'vol_port_hum']].rename(columns = {'vol_port_hum': 'y'})
vol_non_port_hum = fact_table[['ds', 'vol_non_port_hum']].rename(columns = {'vol_non_port_hum': 'y'})

# Other - Gizmo
vol_port_gizmo = fact_table[['ds', 'vol_port_gizmo']].rename(columns = {'vol_port_gizmo': 'y'})
vol_non_port_gizmo = fact_table[['ds', 'vol_non_port_gizmo']].rename(columns = {'vol_non_port_gizmo': 'y'})

# Other - Other
vol_port_other = fact_table[['ds', 'vol_port_other']].rename(columns = {'vol_port_other': 'y'})
vol_non_port_other = fact_table[['ds', 'vol_non_port_other']].rename(columns = {'vol_non_port_other': 'y'})

In [6]:
# Check the individual dataset format (testing only)
vol_port_tablet.head()

,ds,y
0,12/1/2016,112.0
1,12/2/2016,157.0
2,12/3/2016,120.0
3,12/4/2016,99.0
4,12/5/2016,22.0


In [7]:
# Define the holidays
jan_bill_cycle = pd.DataFrame({
    'holiday': 'jan_bill_cycle',
    'ds': pd.to_datetime(['2018-01-05',
                          '2018-01-11',
                          '2018-01-14',
                          '2018-01-17',
                          '2018-01-24',
                          '2018-01-27',
                          '2019-01-29',
                          '2018-01-30',
                          '2018-01-31',
                          '2019-01-05',
                          '2019-01-11',
                          '2019-01-14',
                          '2019-01-17',
                          '2019-01-24',
                          '2019-01-27',
                          '2019-01-29',
                          '2019-01-30',
                          '2019-01-31',
                          '2020-01-05',
                          '2020-01-11',
                          '2020-01-14',
                          '2020-01-17',
                          '2020-01-24',
                          '2020-01-27',
                          '2020-01-29',
                          '2020-01-30',
                          '2020-01-31']),
    'lower_window': 0,
    'upper_window': 0,
})

feb_bill_cycle = pd.DataFrame({
    'holiday': 'feb_bill_cycle',
    'ds': pd.to_datetime(['2018-02-05',
                          '2018-02-11',
                          '2018-02-14',
                          '2018-02-17',
                          '2018-02-24',
                          '2018-02-27',
                          '2019-02-05',
                          '2019-02-11',
                          '2019-02-14',
                          '2019-02-17',
                          '2019-02-24',
                          '2019-02-27',
                          '2020-02-05',
                          '2020-02-11',
                          '2020-02-14',
                          '2020-02-17',
                          '2020-02-24',
                          '2020-02-27',
                          '2020-02-29']),
    'lower_window': 0,
    'upper_window': 0,
})

mar_bill_cycle = pd.DataFrame({
    'holiday': 'mar_bill_cycle',
    'ds': pd.to_datetime(['2018-03-05',
                          '2018-03-11',
                          '2018-03-14',
                          '2018-03-17',
                          '2018-03-24',
                          '2018-03-27',
                          '2019-03-29',
                          '2018-03-30',
                          '2018-03-31',
                          '2019-03-05',
                          '2019-03-11',
                          '2019-03-14',
                          '2019-03-17',
                          '2019-03-24',
                          '2019-03-27',
                          '2019-03-29',
                          '2019-03-30',
                          '2019-03-31',
                          '2020-03-05',
                          '2020-03-11',
                          '2020-03-14',
                          '2020-03-17',
                          '2020-03-24',
                          '2020-03-27',
                          '2020-03-29',
                          '2020-03-30',
                          '2020-03-31']),
    'lower_window': 0,
    'upper_window': 0,
})

apr_bill_cycle = pd.DataFrame({
    'holiday': 'apr_bill_cycle',
    'ds': pd.to_datetime(['2018-04-05',
                          '2018-04-11',
                          '2018-04-14',
                          '2018-04-17',
                          '2018-04-24',
                          '2018-04-27',
                          '2019-04-29',
                          '2018-04-30',
                          '2019-04-05',
                          '2019-04-11',
                          '2019-04-14',
                          '2019-04-17',
                          '2019-04-24',
                          '2019-04-27',
                          '2019-04-29',
                          '2019-04-30',
                          '2020-04-05',
                          '2020-04-11',
                          '2020-04-14',
                          '2020-04-17',
                          '2020-04-24',
                          '2020-04-27',
                          '2020-04-29',
                          '2020-04-30']),
    'lower_window': 0,
    'upper_window': 0,
})

may_bill_cycle = pd.DataFrame({
    'holiday': 'may_bill_cycle',
    'ds': pd.to_datetime(['2018-05-05',
                          '2018-05-11',
                          '2018-05-14',
                          '2018-05-17',
                          '2018-05-24',
                          '2018-05-27',
                          '2019-05-29',
                          '2018-05-30',
                          '2018-05-31',
                          '2019-05-05',
                          '2019-05-11',
                          '2019-05-14',
                          '2019-05-17',
                          '2019-05-24',
                          '2019-05-27',
                          '2019-05-29',
                          '2019-05-30',
                          '2019-05-31',
                          '2020-05-05',
                          '2020-05-11',
                          '2020-05-14',
                          '2020-05-17',
                          '2020-05-24',
                          '2020-05-27',
                          '2020-05-29',
                          '2020-05-30',
                          '2020-05-31']),
    'lower_window': 0,
    'upper_window': 0,
})

jun_bill_cycle = pd.DataFrame({
    'holiday': 'jun_bill_cycle',
    'ds': pd.to_datetime(['2018-06-05',
                          '2018-06-11',
                          '2018-06-14',
                          '2018-06-17',
                          '2018-06-24',
                          '2018-06-27',
                          '2019-06-29',
                          '2018-06-30',
                          '2019-06-05',
                          '2019-06-11',
                          '2019-06-14',
                          '2019-06-17',
                          '2019-06-24',
                          '2019-06-27',
                          '2019-06-29',
                          '2019-06-30',
                          '2020-06-05',
                          '2020-06-11',
                          '2020-06-14',
                          '2020-06-17',
                          '2020-06-24',
                          '2020-06-27',
                          '2020-06-29',
                          '2020-06-30']),
    'lower_window': 0,
    'upper_window': 0,
})

jul_bill_cycle = pd.DataFrame({
    'holiday': 'jul_bill_cycle',
    'ds': pd.to_datetime(['2018-07-05',
                          '2018-07-11',
                          '2018-07-14',
                          '2018-07-17',
                          '2018-07-24',
                          '2018-07-27',
                          '2019-07-29',
                          '2018-07-30',
                          '2018-07-31',
                          '2019-07-05',
                          '2019-07-11',
                          '2019-07-14',
                          '2019-07-17',
                          '2019-07-24',
                          '2019-07-27',
                          '2019-07-29',
                          '2019-07-30',
                          '2019-07-31',
                          '2020-07-05',
                          '2020-07-11',
                          '2020-07-14',
                          '2020-07-17',
                          '2020-07-24',
                          '2020-07-27',
                          '2020-07-29',
                          '2020-07-30',
                          '2020-07-31']),
    'lower_window': 0,
    'upper_window': 0,
})

aug_bill_cycle = pd.DataFrame({
    'holiday': 'aug_bill_cycle',
    'ds': pd.to_datetime(['2018-08-05',
                          '2018-08-11',
                          '2018-08-14',
                          '2018-08-17',
                          '2018-08-24',
                          '2018-08-27',
                          '2019-08-29',
                          '2018-08-30',
                          '2018-08-31',
                          '2019-08-05',
                          '2019-08-11',
                          '2019-08-14',
                          '2019-08-17',
                          '2019-08-24',
                          '2019-08-27',
                          '2019-08-29',
                          '2019-08-30',
                          '2019-08-31',
                          '2020-08-05',
                          '2020-08-11',
                          '2020-08-14',
                          '2020-08-17',
                          '2020-08-24',
                          '2020-08-27',
                          '2020-08-29',
                          '2020-08-30',
                          '2020-08-31']),
    'lower_window': 0,
    'upper_window': 0,
})

sep_bill_cycle = pd.DataFrame({
    'holiday': 'sep_bill_cycle',
    'ds': pd.to_datetime(['2018-09-05',
                          '2018-09-11',
                          '2018-09-14',
                          '2018-09-17',
                          '2018-09-24',
                          '2018-09-27',
                          '2019-09-29',
                          '2018-09-30',
                          '2019-09-05',
                          '2019-09-11',
                          '2019-09-14',
                          '2019-09-17',
                          '2019-09-24',
                          '2019-09-27',
                          '2019-09-29',
                          '2019-09-30',
                          '2020-09-05',
                          '2020-09-11',
                          '2020-09-14',
                          '2020-09-17',
                          '2020-09-24',
                          '2020-09-27',
                          '2020-09-29',
                          '2020-09-30']),
    'lower_window': 0,
    'upper_window': 0,
})

oct_bill_cycle = pd.DataFrame({
    'holiday': 'oct_bill_cycle',
    'ds': pd.to_datetime(['2018-10-05',
                          '2018-10-11',
                          '2018-10-14',
                          '2018-10-17',
                          '2018-10-24',
                          '2018-10-27',
                          '2019-10-29',
                          '2018-10-30',
                          '2018-10-31',
                          '2019-10-05',
                          '2019-10-11',
                          '2019-10-14',
                          '2019-10-17',
                          '2019-10-24',
                          '2019-10-27',
                          '2019-10-29',
                          '2019-10-30',
                          '2019-10-31',
                          '2020-10-05',
                          '2020-10-11',
                          '2020-10-14',
                          '2020-10-17',
                          '2020-10-24',
                          '2020-10-27',
                          '2020-10-29',
                          '2020-10-30',
                          '2020-10-31']),
    'lower_window': 0,
    'upper_window': 0,
})

nov_bill_cycle = pd.DataFrame({
    'holiday': 'nov_bill_cycle',
    'ds': pd.to_datetime(['2018-11-05',
                          '2018-11-11',
                          '2018-11-14',
                          '2018-11-17',
                          '2018-11-24',
                          '2018-11-27',
                          '2019-11-29',
                          '2018-11-30',
                          '2019-11-05',
                          '2019-11-11',
                          '2019-11-14',
                          '2019-11-17',
                          '2019-11-24',
                          '2019-11-27',
                          '2019-11-29',
                          '2019-11-30',
                          '2020-11-05',
                          '2020-11-11',
                          '2020-11-14',
                          '2020-11-17',
                          '2020-11-24',
                          '2020-11-27',
                          '2020-11-29',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

dec_bill_cycle = pd.DataFrame({
    'holiday': 'dec_bill_cycle',
    'ds': pd.to_datetime(['2018-12-05',
                          '2018-12-11',
                          '2018-12-14',
                          '2018-12-17',
                          '2018-12-24',
                          '2018-12-27',
                          '2019-12-29',
                          '2018-12-30',
                          '2018-12-31',
                          '2019-12-05',
                          '2019-12-11',
                          '2019-12-14',
                          '2019-12-17',
                          '2019-12-24',
                          '2019-12-27',
                          '2019-12-29',
                          '2019-12-30',
                          '2019-12-31',
                          '2020-12-05',
                          '2020-12-11',
                          '2020-12-14',
                          '2020-12-17',
                          '2020-12-24',
                          '2020-12-27',
                          '2020-12-29',
                          '2020-12-30',
                          '2020-12-31']),
    'lower_window': 0,
    'upper_window': 0,
})

In [8]:
holidays = pd.concat((jan_bill_cycle,
                      feb_bill_cycle,
                      mar_bill_cycle,
                      apr_bill_cycle,
                      may_bill_cycle,
                      jun_bill_cycle,
                      jul_bill_cycle,
                      aug_bill_cycle,
                      sep_bill_cycle,
                      oct_bill_cycle,
                      nov_bill_cycle,
                      dec_bill_cycle))

In [9]:
# Define and fit the models
# Phones - Basic CDMA
vol_port_basic_cdma_model = Prophet(holidays = holidays).fit(vol_port_basic_cdma)
vol_non_port_basic_cdma_model = Prophet(holidays = holidays).fit(vol_non_port_basic_cdma)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_model = Prophet(holidays = holidays).fit(vol_port_basic_non_cdma)
vol_non_port_basic_non_cdma_model = Prophet(holidays = holidays).fit(vol_non_port_basic_non_cdma)

# Phones - Android & Other
vol_port_sp_android_other_model = Prophet(holidays = holidays).fit(vol_port_sp_android_other)
vol_non_port_sp_android_other_model = Prophet(holidays = holidays).fit(vol_non_port_sp_android_other)

# Phones - Apple
vol_port_sp_apple_model = Prophet(holidays = holidays).fit(vol_port_sp_apple)
vol_non_port_sp_apple_model = Prophet(holidays = holidays).fit(vol_non_port_sp_apple)

# Tablets
vol_port_tablet_model = Prophet(holidays = holidays).fit(vol_port_tablet)
vol_non_port_tablet_model = Prophet(holidays = holidays).fit(vol_non_port_tablet)

# Wearables - Watches
vol_port_watch_model = Prophet(holidays = holidays).fit(vol_port_watch)
vol_non_port_watch_model = Prophet(holidays = holidays).fit(vol_non_port_watch)

# Other - MBB
vol_port_mbb_model = Prophet(holidays = holidays).fit(vol_port_mbb)
vol_non_port_mbb_model = Prophet(holidays = holidays).fit(vol_non_port_mbb)

# Other - HPC
vol_port_hpc_model = Prophet(holidays = holidays).fit(vol_port_hpc)
vol_non_port_hpc_model = Prophet(holidays = holidays).fit(vol_non_port_hpc)

# Other - Hum
vol_port_hum_model = Prophet(holidays = holidays).fit(vol_port_hum)
vol_non_port_hum_model = Prophet(holidays = holidays).fit(vol_non_port_hum)

# Wearables - Gizmo
vol_port_gizmo_model = Prophet(holidays = holidays).fit(vol_port_gizmo)
vol_non_port_gizmo_model = Prophet(holidays = holidays).fit(vol_non_port_gizmo)

# Other - Other
vol_port_other_model = Prophet(holidays = holidays).fit(vol_port_other)
vol_non_port_other_model = Prophet(holidays = holidays).fit(vol_non_port_other)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily

In [10]:
# Number of periods into the future to forecast
n = 155

In [11]:
# Create dataframes to hold predictions
# Phones - Basic CDMA
vol_port_basic_cdma_future = vol_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_cdma_future = vol_non_port_basic_cdma_model.make_future_dataframe(periods = n)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_future = vol_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_non_cdma_future = vol_non_port_basic_non_cdma_model.make_future_dataframe(periods = n)

# Phones - Android & Other
vol_port_sp_android_other_future = vol_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_non_port_sp_android_other_future = vol_non_port_sp_android_other_model.make_future_dataframe(periods = n)

# Phones - Apple
vol_port_sp_apple_future = vol_port_sp_apple_model.make_future_dataframe(periods = n)
vol_non_port_sp_apple_future = vol_non_port_sp_apple_model.make_future_dataframe(periods = n)

# Tablets
vol_port_tablet_future = vol_port_tablet_model.make_future_dataframe(periods = n)
vol_non_port_tablet_future = vol_non_port_tablet_model.make_future_dataframe(periods = n)

# Wearables - Watches
vol_port_watch_future = vol_port_watch_model.make_future_dataframe(periods = n)
vol_non_port_watch_future = vol_non_port_watch_model.make_future_dataframe(periods = n)

# Other - MBB
vol_port_mbb_future = vol_port_mbb_model.make_future_dataframe(periods = n)
vol_non_port_mbb_future = vol_non_port_mbb_model.make_future_dataframe(periods = n)

# Other - HPC
vol_port_hpc_future = vol_port_hpc_model.make_future_dataframe(periods = n)
vol_non_port_hpc_future = vol_non_port_hpc_model.make_future_dataframe(periods = n)

# Other - Hum
vol_port_hum_future = vol_port_hum_model.make_future_dataframe(periods = n)
vol_non_port_hum_future = vol_non_port_hum_model.make_future_dataframe(periods = n)

# Wearables - Gizmo
vol_port_gizmo_future = vol_port_gizmo_model.make_future_dataframe(periods = n)
vol_non_port_gizmo_future = vol_non_port_gizmo_model.make_future_dataframe(periods = n)

# Other - Other
vol_port_other_future = vol_port_other_model.make_future_dataframe(periods = n)
vol_non_port_other_future = vol_non_port_other_model.make_future_dataframe(periods = n)

In [12]:
# Make predictions
# Phones - Basic CDMA
vol_port_basic_cdma_forecast = vol_port_basic_cdma_model.predict(vol_port_basic_cdma_future)
vol_non_port_basic_cdma_forecast = vol_non_port_basic_cdma_model.predict(vol_non_port_basic_cdma_future)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_forecast = vol_port_basic_non_cdma_model.predict(vol_port_basic_non_cdma_future)
vol_non_port_basic_non_cdma_forecast = vol_non_port_basic_non_cdma_model.predict(vol_non_port_basic_non_cdma_future)

# Phones - Android & Other
vol_port_sp_android_other_forecast = vol_port_sp_android_other_model.predict(vol_port_sp_android_other_future)
vol_non_port_sp_android_other_forecast = vol_non_port_sp_android_other_model.predict(vol_non_port_sp_android_other_future)

# Phones - Apple
vol_port_sp_apple_forecast = vol_port_sp_apple_model.predict(vol_port_sp_apple_future)
vol_non_port_sp_apple_forecast = vol_non_port_sp_apple_model.predict(vol_non_port_sp_apple_future)

# Tablets
vol_port_tablet_forecast = vol_port_tablet_model.predict(vol_port_tablet_future)
vol_non_port_tablet_forecast = vol_non_port_tablet_model.predict(vol_non_port_tablet_future)

# Wearables - Watches
vol_port_watch_forecast = vol_port_watch_model.predict(vol_port_watch_future)
vol_non_port_watch_forecast = vol_non_port_watch_model.predict(vol_non_port_watch_future)

# Other - MBB
vol_port_mbb_forecast = vol_port_mbb_model.predict(vol_port_mbb_future)
vol_non_port_mbb_forecast = vol_non_port_mbb_model.predict(vol_non_port_mbb_future)

# Other - HPC
vol_port_hpc_forecast = vol_port_hpc_model.predict(vol_port_hpc_future)
vol_non_port_hpc_forecast = vol_non_port_hpc_model.predict(vol_non_port_hpc_future)

# Other - Hum
vol_port_hum_forecast = vol_port_hum_model.predict(vol_port_hum_future)
vol_non_port_hum_forecast = vol_non_port_hum_model.predict(vol_non_port_hum_future)

# Wearables - Gizmo
vol_port_gizmo_forecast = vol_port_gizmo_model.predict(vol_port_gizmo_future)
vol_non_port_gizmo_forecast = vol_non_port_gizmo_model.predict(vol_non_port_gizmo_future)

# Other - Other
vol_port_other_forecast = vol_port_other_model.predict(vol_port_other_future)
vol_non_port_other_forecast = vol_non_port_other_model.predict(vol_non_port_other_future)

In [13]:
# Get the datestamp and prediction columns
# Phones - Basic CDMA
vol_port_basic_cdma_forecast_subset = vol_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_CDMA'})
vol_non_port_basic_cdma_forecast_subset = vol_non_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_CDMA'})

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_forecast_subset = vol_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_non_CDMA'})
vol_non_port_basic_non_cdma_forecast_subset = vol_non_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_non_CDMA'})

# Phones - Android & Other
vol_port_sp_android_other_forecast_subset = vol_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Android_Other'})
vol_non_port_sp_android_other_forecast_subset = vol_non_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Android_Other'})

# Phones - Apple
vol_port_sp_apple_forecast_subset = vol_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Apple'})
vol_non_port_sp_apple_forecast_subset = vol_non_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Apple'})

# Tablets
vol_port_tablet_forecast_subset = vol_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Tablet'})
vol_non_port_tablet_forecast_subset = vol_non_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Tablet'})

# Wearables - Watches
vol_port_watch_forecast_subset = vol_port_watch_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Watch'})
vol_non_port_watch_forecast_subset = vol_non_port_watch_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Watch'})

# Other - MBB
vol_port_mbb_forecast_subset = vol_port_mbb_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_MBB'})
vol_non_port_mbb_forecast_subset = vol_non_port_mbb_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_MBB'})

# Other - HPC
vol_port_hpc_forecast_subset = vol_port_hpc_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_HPC'})
vol_non_port_hpc_forecast_subset = vol_non_port_hpc_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_HPC'})

# Other - Hum
vol_port_hum_forecast_subset = vol_port_hum_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Hum'})
vol_non_port_hum_forecast_subset = vol_non_port_hum_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Hum'})

# Wearables - Gizmo
vol_port_gizmo_forecast_subset = vol_port_gizmo_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Gizmo'})
vol_non_port_gizmo_forecast_subset = vol_non_port_gizmo_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Gizmo'})

# Other - Other
vol_port_other_forecast_subset = vol_port_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Other'})
vol_non_port_other_forecast_subset = vol_non_port_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Other'})

In [14]:
# Check the format of the subsetted forecasts (testing only)
vol_non_port_tablet_forecast_subset.tail()

,ds,Vol_Non_Port_Tablet
1487,2020-12-27,-897.519171
1488,2020-12-28,3457.929820
1489,2020-12-29,-850.281268
1490,2020-12-30,-713.259695
1491,2020-12-31,-719.966863


In [15]:
# Growth multiplier
gm = 1.00

In [16]:
# Replace any negative values and scale the forecast using the growth multiplier
# Phones - Basic CDMA
vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'] = vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'].abs() * gm
vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'] = vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'].abs() * gm

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'] = vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'].abs() * gm
vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'] = vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'].abs() * gm

# Phones - Android & Other
vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'] = vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'].abs() * gm
vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'] = vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'].abs() * gm

# Phones - Apple
vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'] = vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'].abs() * gm
vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'] = vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'].abs() * gm

# Tablets
vol_port_tablet_forecast_subset['Vol_Port_Tablet'] = vol_port_tablet_forecast_subset['Vol_Port_Tablet'].abs() * gm
vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'] = vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'].abs() * gm

# Wearables - Watches
vol_port_watch_forecast_subset['Vol_Port_Watch'] = vol_port_watch_forecast_subset['Vol_Port_Watch'].abs() * gm
vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'] = vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'].abs() * gm

# Other - MBB
vol_port_mbb_forecast_subset['Vol_Port_MBB'] = vol_port_mbb_forecast_subset['Vol_Port_MBB'].abs() * gm
vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'] = vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'].abs() * gm

# Other - HPC
vol_port_hpc_forecast_subset['Vol_Port_HPC'] = vol_port_hpc_forecast_subset['Vol_Port_HPC'].abs() * gm
vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'] = vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'].abs() * gm

# Other - Hum
vol_port_hum_forecast_subset['Vol_Port_Hum'] = vol_port_hum_forecast_subset['Vol_Port_Hum'].abs() * gm
vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'] = vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'].abs() * gm

# Wearables - Gizmo
vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'] = vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'].abs() * gm
vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'] = vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'].abs() * gm

# Other - Other
vol_port_other_forecast_subset['Vol_Port_Other'] = vol_port_other_forecast_subset['Vol_Port_Other'].abs() * gm
vol_non_port_other_forecast_subset['Vol_Non_Port_Other'] = vol_non_port_other_forecast_subset['Vol_Non_Port_Other'].abs() * gm

In [17]:
# Check the format of the adjusted forecasts (testing only)
vol_non_port_tablet_forecast_subset.tail()

,ds,Vol_Non_Port_Tablet
1487,2020-12-27,897.519171
1488,2020-12-28,3457.929820
1489,2020-12-29,850.281268
1490,2020-12-30,713.259695
1491,2020-12-31,719.966863


In [18]:
# Combine the forecasts into one table
# Voluntary Port Outs - Phones
discos_device_grp_pred_table = vol_port_basic_cdma_forecast_subset
discos_device_grp_pred_table.insert(2, 'Vol_Port_Basic_non_CDMA', vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(3, 'Vol_Port_SP_Android_Other', vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(4, 'Vol_Port_SP_Apple', vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'])

# Voluntary Non Port Outs - Phones
discos_device_grp_pred_table.insert(5, 'Vol_Non_Port_Basic_CDMA', vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'])
discos_device_grp_pred_table.insert(6, 'Vol_Non_Port_Basic_non_CDMA', vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(7, 'Vol_Non_Port_SP_Android_Other', vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(8, 'Vol_Non_Port_SP_Apple', vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'])

# Tablets
discos_device_grp_pred_table.insert(9, 'Vol_Port_Tablet', vol_port_tablet_forecast_subset['Vol_Port_Tablet'])
discos_device_grp_pred_table.insert(10, 'Vol_Non_Port_Tablet', vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'])

# Wearables - Watches
discos_device_grp_pred_table.insert(11, 'Vol_Port_Watch', vol_port_watch_forecast_subset['Vol_Port_Watch'])
discos_device_grp_pred_table.insert(12, 'Vol_Non_Port_Watch', vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'])

# Other - MBB
discos_device_grp_pred_table.insert(13, 'Vol_Port_MBB', vol_port_mbb_forecast_subset['Vol_Port_MBB'])
discos_device_grp_pred_table.insert(14, 'Vol_Non_Port_MBB', vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'])

# Other - HPC
discos_device_grp_pred_table.insert(15, 'Vol_Port_HPC', vol_port_hpc_forecast_subset['Vol_Port_HPC'])
discos_device_grp_pred_table.insert(16, 'Vol_Non_Port_HPC', vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'])

# Other - Hum
discos_device_grp_pred_table.insert(17, 'Vol_Port_Hum', vol_port_hum_forecast_subset['Vol_Port_Hum'])
discos_device_grp_pred_table.insert(18, 'Vol_Non_Port_Hum', vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'])

# Wearables - Gizmo
discos_device_grp_pred_table.insert(19, 'Vol_Port_Gizmo', vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'])
discos_device_grp_pred_table.insert(20, 'Vol_Non_Port_Gizmo', vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'])

# Other - Other
discos_device_grp_pred_table.insert(21, 'Vol_Port_Other', vol_port_other_forecast_subset['Vol_Port_Other'])
discos_device_grp_pred_table.insert(22, 'Vol_Non_Port_Other', vol_non_port_other_forecast_subset['Vol_Non_Port_Other'])

In [19]:
# Check the format of the final predictions table (testing only)
discos_device_grp_pred_table.tail()

,ds,Vol_Port_Basic_CDMA,Vol_Port_Basic_non_CDMA,Vol_Port_SP_Android_Other,Vol_Port_SP_Apple,Vol_Non_Port_Basic_CDMA,Vol_Non_Port_Basic_non_CDMA,Vol_Non_Port_SP_Android_Other,Vol_Non_Port_SP_Apple,Vol_Port_Tablet,Vol_Non_Port_Tablet,Vol_Port_Watch,Vol_Non_Port_Watch,Vol_Port_MBB,Vol_Non_Port_MBB,Vol_Port_HPC,Vol_Non_Port_HPC,Vol_Port_Hum,Vol_Non_Port_Hum,Vol_Port_Gizmo,Vol_Non_Port_Gizmo,Vol_Port_Other,Vol_Non_Port_Other
1487,2020-12-27,93.746296,23.645088,261.721222,284.096967,286.231431,84.638326,129.818213,62.329990,10.580715,897.519171,3.303235,274.847008,19.124838,232.275900,12.179185,45.090213,1.777405,260.460822,4.641139,74.728228,16.219926,3.505514
1488,2020-12-28,629.486039,31.039570,3488.675535,5492.595832,267.656339,150.865874,1075.666464,1595.117969,66.074695,3457.929820,15.460078,824.098096,35.574008,891.894850,32.673101,200.012928,1.982417,451.696457,9.842906,409.069604,41.425306,42.657967
1489,2020-12-29,81.709681,28.313117,238.873932,320.779626,275.576787,85.960034,104.203616,98.711249,11.891476,850.281268,2.881069,271.512775,28.923246,226.187910,20.373139,45.857250,1.864738,258.280827,6.288052,86.131460,15.472497,5.007798
1490,2020-12-30,53.752897,28.030611,321.162498,446.942560,239.291256,86.568889,70.917153,159.988755,10.224659,713.259695,4.259466,282.429971,18.065088,191.942055,11.211542,34.496903,1.672211,248.020241,5.059067,94.288448,18.087129,6.235671
1491,2020-12-31,42.695567,21.724128,303.309148,425.187813,233.251081,88.397974,61.879761,174.803201,9.849061,719.966863,4.482143,275.660136,15.914895,188.683477,10.005574,32.271093,1.867747,253.282207,4.833638,94.699674,18.449188,4.683298


In [ ]:
# Calculate the root mean squared error (testing only)
print("Vol Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_CDMA'] - fact_table['vol_port_basic_cdma']) ** 2)))
print("Vol Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_non_CDMA'] - fact_table['vol_port_basic_non_cdma']) ** 2)))
print("Vol Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Android_Other'] - fact_table['vol_port_sp_android_other']) ** 2)))
print("Vol Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Apple'] - fact_table['vol_port_sp_apple']) ** 2)))
print("Vol Non Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_CDMA'] - fact_table['vol_non_port_basic_cdma']) ** 2)))
print("Vol Non Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_non_CDMA'] - fact_table['vol_non_port_basic_non_cdma']) ** 2)))
print("Vol Non Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Android_Other'] - fact_table['vol_non_port_sp_android_other']) ** 2)))
print("Vol Non Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Apple'] - fact_table['vol_non_port_sp_apple']) ** 2)))
print("Vol Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Tablet'] - fact_table['vol_port_tablet']) ** 2)))
print("Vol Non Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Tablet'] - fact_table['vol_non_port_tablet']) ** 2)))

Vol Port Basic CDMA RMSE: 300.990941
Vol Port Basic Non CDMA RMSE: 17.416380
Vol Port SP Android Other RMSE: 1194.310291
Vol Port SP Apple RMSE: 1459.460262
Vol Non Port Basic CDMA RMSE: 386.307813
Vol Non Port Basic Non CDMA RMSE: 17.876134
Vol Non Port SP Android Other RMSE: 477.962450
Vol Non Port SP Apple RMSE: 474.977620
Vol Port Tablet RMSE: 31.740352
Vol Non Port Tablet RMSE: 1688.689850


In [20]:
# Save the predictions table to a csv file
discos_device_grp_pred_table.to_csv('drive/My Drive/Churn/discos_device_grp_future_demand_aug_pred_table.csv')